In [1]:
import os
import numpy as np
import re

In [2]:
## Download glove embedding (reference .. https://edumunozsala.github.io/BlogEms/jupyter/nlp/classification/embeddings/python/2020/08/15/Intro_NLP_WordEmbeddings_Classification.html)
embedding_file = "../embedding/glove.6B/glove.6B.300d.txt"

In [3]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(embedding_file)
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [4]:
import pandas as pd

In [5]:
def get_edited_headline(row):
    expr = re.compile("<(.*?)/>")
    edited_headline = (re.sub(expr,row.edit,row.original)).lower()
    #print((edited_headline))
    return edited_headline

def get_context(row):
    
    expr = re.compile("<(.*?)/>")
    context = (re.sub(expr,' ',row.original)).lower()
    #print((edited_headline))
    return context

def make_original_string(row):
    s=row.original
    expr = re.compile("<")
    s=re.sub(expr,'',s).lower()
    expr = re.compile(">")
    s=re.sub(expr,'',s).lower()
    return s

def enhance_df(df):
    df['edited'] = df.apply(get_edited_headline,axis=1)
    df['edited_len'] = df.apply(lambda row:len(row.edited.split(' ')),axis=1)
    df['context'] = df.apply(get_context,axis=1)
    df['original_string']=df.apply(make_original_string,axis=1)
    df['edit'] = df.edit.str.lower()
    return df

In [6]:
train_df = pd.read_csv("./data/semeval_train.csv")

train_df = enhance_df(train_df)
train_df.head()


,id,original,edit,grades,meanGrade,edited,edited_len,context,original_string
0,5515,Trump <Lawyers/> Want A Second Special Counsel,toupees,32110,1.4,trump toupees want a second special counsel,7,trump want a second special counsel,trump lawyers/ want a second special counsel
1,12469,One industry suddenly has ' unfettered access ...,pie,21110,1.0,one industry suddenly has ' unfettered access ...,22,one industry suddenly has ' unfettered access ...,one industry suddenly has ' unfettered access ...
2,10947,"After healthcare vote , California Rep. Jeff D...",wrote,10000,0.2,"after healthcare vote , california rep. jeff d...",19,"after healthcare vote , california rep. jeff d...","after healthcare vote , california rep. jeff d..."
3,5518,Since when was there such a low bar for <defam...,planetary,10000,0.2,since when was there such a low bar for planet...,18,since when was there such a low bar for sett...,since when was there such a low bar for defama...
4,11453,"Donald Trump Unfollowed Reince Priebus , The U...",compliment,32221,2.0,"donald trump unfollowed reince priebus , the u...",13,"donald trump unfollowed reince priebus , the u...","donald trump unfollowed reince priebus , the u..."


In [7]:
test_df = pd.read_csv("./data/semeval_test.csv")
test_df = enhance_df(test_df)

In [8]:
from keras.preprocessing.text import Tokenizer

In [9]:
docs = train_df.edited
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [10]:
print(vocab_size)

9925


In [11]:
train_df['edited_len'].describe()

count    7721.000000
mean       12.406165
std         3.632329
min         4.000000
25%        10.000000
50%        12.000000
75%        15.000000
max        26.000000
Name: edited_len, dtype: float64

In [12]:
max_len = 20

In [13]:
train_df

,id,original,edit,grades,meanGrade,edited,edited_len,context,original_string
0,5515,Trump <Lawyers/> Want A Second Special Counsel,toupees,32110,1.4,trump toupees want a second special counsel,7,trump want a second special counsel,trump lawyers/ want a second special counsel
1,12469,One industry suddenly has ' unfettered access ...,pie,21110,1.0,one industry suddenly has ' unfettered access ...,22,one industry suddenly has ' unfettered access ...,one industry suddenly has ' unfettered access ...
2,10947,"After healthcare vote , California Rep. Jeff D...",wrote,10000,0.2,"after healthcare vote , california rep. jeff d...",19,"after healthcare vote , california rep. jeff d...","after healthcare vote , california rep. jeff d..."
3,5518,Since when was there such a low bar for <defam...,planetary,10000,0.2,since when was there such a low bar for planet...,18,since when was there such a low bar for sett...,since when was there such a low bar for defama...
4,11453,"Donald Trump Unfollowed Reince Priebus , The U...",compliment,32221,2.0,"donald trump unfollowed reince priebus , the u...",13,"donald trump unfollowed reince priebus , the u...","donald trump unfollowed reince priebus , the u..."
...,...,...,...,...,...,...,...,...,...
7716,10954,If Trump wants to use this memo to fire Rosens...,tweeting,32111,1.6,if trump wants to use this memo to fire rosens...,20,if trump wants to use this memo to fire rosens...,if trump wants to use this memo to fire rosens...
7717,10932,Russia Will Test ' Unstoppable ' Satan <Missil...,prayer,33210,1.8,russia will test ' unstoppable ' satan prayer ...,15,russia will test ' unstoppable ' satan by en...,russia will test ' unstoppable ' satan missile...
7718,8303,Trump campaign had contact with Russian <intel...,vodka,21111,1.2,trump campaign had contact with russian vodka ...,9,trump campaign had contact with russian : nyt,trump campaign had contact with russian intell...
7719,9277,How Trump 's Twitter account is fueling a GOP ...,electricity,21100,0.8,how trump 's twitter account is fueling a gop ...,11,how trump 's twitter account is fueling a gop ...,how trump 's twitter account is fueling a gop ...


In [14]:
def compute_average_embedding(sentence):
    embeddings = []
    for word in sentence:
        if word in embeddings_index:
            embeddings.append(np.asarray(embeddings_index[word]))
    if len(embeddings) == 0:
        print(sentence)
    mean_embedding = np.mean(embeddings,axis=0)
    #print(mean_embedding.shape)
    return mean_embedding
    

In [15]:
def compute_embedding_1(df):
    '''Dot product of both sentence averages'''
    edited_sentences = df.edited
    edited_embeddings = np.asarray([compute_average_embedding(sentence) for sentence in edited_sentences])
    
    original_sentences = df.original_string
    original_embeddings = np.asarray([compute_average_embedding(sentence) for sentence in original_sentences])
    
    #return (edited_embeddings * original_embeddings)
    return edited_embeddings

    
    

In [16]:
train_embeddings = compute_embedding_1(train_df)
test_embeddings = compute_embedding_1(test_df)

In [17]:
labels = train_df.meanGrade.values

In [18]:
train_embeddings.shape

(7721, 300)

In [19]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Input
import keras

In [20]:
# define model
model = Sequential()
 
model.add(Input(shape=(300)))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
#model.add(Flatten())
model.add(Dense(1))
# compile the model
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                9632      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 10,721
Trainable params: 10,721
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.fit(train_embeddings, labels, epochs=5, verbose=1)


Epoch 1/5
242/242 [==============================] - 1s 853us/step - loss: 0.3490
Epoch 2/5
242/242 [==============================] - 0s 842us/step - loss: 0.3456
Epoch 3/5
242/242 [==============================] - 0s 790us/step - loss: 0.3384
Epoch 4/5
242/242 [==============================] - 0s 793us/step - loss: 0.3370
Epoch 5/5
242/242 [==============================] - 0s 816us/step - loss: 0.3459


In [22]:
test_gt = test_df.meanGrade.values

In [23]:
test_pred = model.predict(test_embeddings)

In [24]:
def evaluate_predictions(pred_df):
    '''RMSE is the metric used to compare predicted score and actual score'''
    diff = (pred_df.meanGrade - pred_df.pred).values
    mean_squared_diff = np.sum(diff*diff)/diff.shape[0]
    rmse = np.sqrt(mean_squared_diff)
    return rmse

In [25]:
test_df['pred']= test_pred

In [26]:
evaluate_predictions(test_df)

0.5813892694721935

In [27]:
train_pred = model.predict(train_embeddings)

In [28]:
train_df['pred'] = train_pred

In [29]:
evaluate_predictions(train_df)

0.5792917711474913

In [30]:
train_df = pd.read_csv("./data/semeval_train.csv")

train_df = enhance_df(train_df)
train_df.head()

,id,original,edit,grades,meanGrade,edited,edited_len,context,original_string
0,5515,Trump <Lawyers/> Want A Second Special Counsel,toupees,32110,1.4,trump toupees want a second special counsel,7,trump want a second special counsel,trump lawyers/ want a second special counsel
1,12469,One industry suddenly has ' unfettered access ...,pie,21110,1.0,one industry suddenly has ' unfettered access ...,22,one industry suddenly has ' unfettered access ...,one industry suddenly has ' unfettered access ...
2,10947,"After healthcare vote , California Rep. Jeff D...",wrote,10000,0.2,"after healthcare vote , california rep. jeff d...",19,"after healthcare vote , california rep. jeff d...","after healthcare vote , california rep. jeff d..."
3,5518,Since when was there such a low bar for <defam...,planetary,10000,0.2,since when was there such a low bar for planet...,18,since when was there such a low bar for sett...,since when was there such a low bar for defama...
4,11453,"Donald Trump Unfollowed Reince Priebus , The U...",compliment,32221,2.0,"donald trump unfollowed reince priebus , the u...",13,"donald trump unfollowed reince priebus , the u...","donald trump unfollowed reince priebus , the u..."


In [31]:
test_df['diff'] = np.abs(test_df.pred - test_df.meanGrade)


test_df=test_df.sort_values(by='diff')

head=test_df.head()

tail=test_df.tail()

## Low errors
for i,row in head.iterrows():
    print(row.edited)
    print(row.original)
    print(row.meanGrade,row.pred)

u.s. , south korea revise trade deal , korean wrestler faces quota
U.S. , South Korea revise trade deal , Korean <steel/> faces quota
0.8 0.8001803755760193
gop senators break glasses to protect mueller from trump
GOP senators break <ranks/> to protect Mueller from Trump
1.0 0.9979852437973022
californians paid babysitters extra : the state assembly should investigate at&amp;t ’s cross-subsidies .
Californians Paid <Billions/> Extra : The State Assembly Should Investigate AT&amp;T ’s Cross-Subsidies .
0.8 0.7977107167243958
cia director warns iranian general on carpets 
CIA director warns Iranian general on <Iraq/> 
0.8 0.8028097748756409
trump says he may pull sunshine from california
Trump Says He May Pull <Immigration Enforcement/> From California
1.0 0.9960620403289795


In [32]:
# High Errors

for i,row in tail.iterrows():
    print(row.edited)
    print(row.original)
    print(row.meanGrade,row.pred)

us navy ship fired tequila shots at an iranian boat in the persian gulf
US Navy ship fired <warning/> shots at an Iranian boat in the Persian Gulf
2.6 0.8740834593772888
missouri gov. eric greitens admits to affair , denies enjoyment 
Missouri Gov. Eric Greitens admits to affair , denies <blackmail/> 
2.6 0.8334009051322937
trump admits tariffs could cause ' pain ' in buttock 
Trump admits tariffs could cause ' pain ' in <markets/> 
2.8 0.9912739396095276
trump forced sons to wear " very tiny " bathing suits and higher heels after buying beauty pageants
Trump forced <women/> to wear " very tiny " bathing suits and higher heels after buying beauty pageants
2.8 0.9389268755912781
the last president to kiss an fbi director ? bill clinton
The last president to <fire/> an FBI director ? Bill Clinton
2.8 0.8810884356498718
